In [50]:
import torch
import torchvision

import matplotlib.pyplot as plt
%matplotlib inline
import random
import json
from PIL import Image
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
data_path = '../data/2.7万张道路线精细标注抽取2000张/'

# Prepare data

### 1. train val split
### 2. get classes

In [58]:
data = os.listdir(data_path)
names = []
for line in data:
    name, t = line.split('.')
    if t == 'jpg':
        names.append(name)
print('total image:{}'.format(len(names)))

val = random.sample(names,100)
train = list(set(names) - set(val))

print('train val split: train:{}, val:{}'.format(len(val), len(train)))

classes = set()
for name in names:
    json_path = data_path + name + '.json'
    annotation = json.load(open(json_path))
    annotation = annotation['datalist']
    for mask in annotation:
        classes.add(mask['type'])
classes = [(c,idx) for idx,c in enumerate(classes,1)]

print('total class:{}'.format(len(classes)))
print(classes)

data_json = {
    'train':train,
    'val':val,
    'classes':classes
}

save_path = data_path+'train_val.json'
json.dump(train_val_json, open(save_path,'w'))
print('save to: '+ save_path)



total image:2000
train val split: train:100, val:1900
total class:21
[('seven', 1), ('stop line', 2), ('turn right', 3), ('three', 4), ('turn left', 5), ('six', 6), ('solid line', 7), ('straight', 8), ('dotted line', 9), ('void', 10), ('four', 11), ('turn around', 12), ('straight and left', 13), ('route void', 14), ('straight and turn around', 15), ('sidewalk', 16), ('straight and right', 17), ('one', 18), ('eight', 19), ('nine', 20), ('zero', 21)]
save to: ../data/2.7万张道路线精细标注抽取2000张/train_val.json


In [41]:
from mrcnn import utils

class CarlaneDataset(utils.Dataset):
    """
    train_val.json:{
        "train": xx,
        "val":xx,
        "classes":xx
    }
    train, val: [image_name,...]
    classes: [(class_name, class_id),...]
    """
    
    def load_carlane(self, dataset_dir, subset):
        assert subset in ["train", "val"]
        data = json.load(open(os.path.join(dataset_dir,'train_val.json'),'r'))
        
        classes = data['classes']
        for c in classes:
            self.add_class("carlane", c[1], c[0])
            
        self.class_map = {}
        for c in classes:
            self.class_map[c[0]] = c[1]
            
        image_names = data[subset]
        
        # Add images
        for image_name in image_names:
            json_path = os.path.join(dataset_dir, image_name + '.json')
            annotation = json.load(open(json_path))
            annotation = annotation['datalist']
            polygons = [ a['arr'] for a in annotation]
            polygons_type = [ a['type'] for a in annotation]
            
            